## Clone Repository

In [17]:
!git clone https://github.com/piainmaster/ProblematicInternetUse.git

Cloning into 'ProblematicInternetUse'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 12 (delta 2), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (12/12), 7.40 KiB | 7.40 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [18]:
%cd ProblematicInternetUse

/content/ProblematicInternetUse/ProblematicInternetUse/ProblematicInternetUse


## Import Dataset

In [19]:
!pip install -q kaggle # install kaggle api client

In [23]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [24]:
!chmod 600 ~/.kaggle/kaggle.json

In [25]:
!kaggle competitions download -c child-mind-institute-problematic-internet-use

100% 6.20G/6.21G [01:15<00:00, 85.4MB/s]
100% 6.21G/6.21G [01:15<00:00, 88.3MB/s]


In [26]:
!unzip -q child-mind-institute-problematic-internet-use.zip -d /content/dataset

## Push changes to GitHub

In [27]:
!git config --global user.email "pia.neuwirth@stud.plus.ac.at"
!git config --global user.name "piainmaster"

In [28]:
!git add Main.ipynb
!git commit -m "Updated the notebook with new changes"

On branch main
Your branch is up to date with 'origin/main'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	child-mind-institute-problematic-internet-use.zip

nothing added to commit but untracked files present (use "git add" to track)


In [ ]:
!git push origin main

## Import Parquet Data

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Import data sets
#train_data = pd.read_parquet("/content/dataset/series_train.parquet")
test_data = pd.read_parquet("/content/dataset/series_test.parquet")

df = test_data
df.to_csv('series_test.csv')

print("Test data shape:" + str(df.shape))
# Check for missing values
print("\nMissing Values in Test Data:")
print(df.isnull().sum())

print(df.head())
print(df.info())